In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Ruta al directorio de las imágenes
image_directory = 'dataset'

In [3]:
# Dimensiones de las imágenes de entrada
img_width, img_height = 150, 150

In [4]:
# Hiperparámetros del modelo
batch_size = 32
epochs = 1

In [5]:
# Preprocesamiento y aumento de datos
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

In [6]:
# Cargar y dividir los datos de entrenamiento y validación
train_data = datagen.flow_from_directory(
    image_directory,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

validation_data = datagen.flow_from_directory(
    image_directory,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

Found 9600 images belonging to 12 classes.
Found 2400 images belonging to 12 classes.


In [7]:
# Construir el modelo de la red neuronal convolucional
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [8]:
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
# Entrenar el modelo
model.fit(train_data, epochs=epochs, validation_data=validation_data)

300/300 [==============================] - 88s 292ms/step - loss: -92575344099328.0000 - accuracy: 0.0832 - val_loss: -813251453517824.0000 - val_accuracy: 0.0833


In [ ]:
# Guardar el modelo entrenado
model.save('fruits_model.h5')